# Pregunta 20

Cantidad de [Stubs](https://en.wikipedia.org/wiki/Wikipedia:Stub) por categoría en la Wikipedia.

> A stub is an article deemed too short to provide encyclopedic coverage of a subject.


### Importante

Se tomara como criterio de Stub:

* El articulo debe tener como maximo 1500 caracteres en el texto principal

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

sqlContext = SQLContext(sc)

21/10/31 20:35:01 WARN Utils: Your hostname, Ideapad3 resolves to a loopback address: 127.0.1.1; using 192.168.0.237 instead (on interface wlp0s20f3)
21/10/31 20:35:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/31 20:35:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/luis/anaconda3/envs/organizacionDatos/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
category_links = sqlContext.read.csv('./Wikipedia_dataset_2c2021/categorylinks.csv', 
                                    header=True, inferSchema=True, multiLine=True)
rdd_category_links = category_links.rdd


texts = pd.read_csv('./Wikipedia_dataset_2c2021/contents_text_sample.csv')
texts.to_parquet('contents_text_sample.parquet')
del texts
contents_text_sample = sqlContext.read.parquet('contents_text_sample.parquet')

rdd_contents_text_sample = contents_text_sample.rdd

In [4]:
# Verifico que lo haya leido bien
rdd_contents_text_sample.take(5)

[Row(id=50, title='Anacardiaceae', text="{{Ficha de taxón\n| name = Anacardiáceas\n| image = Gui1 cashewfruit2.jpg\n| image_caption = ''[[Anacardium occidentale]]''\n| regnum = [[Plantae]]\n| divisio  = [[Magnoliophyta]]\n| classis  = [[Magnoliopsida]]\n| ordo = [[Sapindales]]\n| familia = '''Anacardiaceae'''\n| familia_authority = [[Robert Brown|R.Br.]]\n| subdivision_ranks = Géneros\n| subdivision = \nVer texto\n}}\n'''Anacardiaceae''' es una [[familia (biología)|familia]] de [[Plantae|plantas]] esencialmente [[árbol|arbóreas]] y [[arbusto|arbustivas]] perteneciente al [[Orden (biología)|orden]] [[Sapindales]]. La constituyen 77 [[género (biología)|géneros]] con unas 700 especies aceptadas, de las casi 3000 descritas, propias de países tropicales, cálidos y templados.<ref>[http://www.theplantlist.org/1.1/browse/A/Anacardiaceae/ Anacardiaceae en The Plant List, vers. 1.1, 2013]</ref>\n\n== Descripción ==\nSon árboles, arbustos, raramente subarbustos o [[trepadora]]s, frecuentemente co

## Limpio los datos que tengan texto o id nulo

In [5]:
rdd_contents_text_sample = rdd_contents_text_sample.filter(lambda x: x.text != None and x.id != None)

## Filtro los contenidos Stub

Limpio el texto para que tenga unicamente palabras.

Luego filtro aquellos textos que tengan menos de 1500 caracteres.

In [6]:
rdd_stubs = rdd_contents_text_sample.map(lambda x: (str(x.id), x.text.replace('\n', '')
         .replace('.', '')
         .replace('|','')
         .replace('[','')
         .replace(']','')
         .replace('{','')
        .replace('}','')
        .replace('=','')
        .replace('<','')
        .replace('>','')
        .replace('\'','')
        .replace('ref','')
        .replace('*','')
        .replace('–','')
        )
    ).filter(lambda x: len(x[1]) < 1500) 

In [7]:
# Verifico que obtenga los stubs correctamente
rdd_stubs.take(5)

[('179',
  'Ficha de taxón name  Arundoclaytonia image   image_width  250px image_caption   regnum  Plantae divisio  Magnoliophyta classis  Liliopsida subclassis  Commelinidae ordo   Poales familia  Poaceae subfamilia  Panicoideae tribus  Steyermarkochloeae subtribus   alliance   genus  Arundoclaytonia genus_authority  Davidse & Ellis subdivision_ranks  Especies subdivision  Ver textoArundoclaytonia es un Genusgénero de plantas herbáceas perteneciente a la familia de las poáceas(en inglés) cita weburlhttp://delta-intkeycom/grass/www/arundoclhtm títuloThe grass genera of the world: descriptions, illustrations, identification, and information retrieval; including synonyms, morphology, anatomy, physiology, phytochemistry, cytology, classification, pathogens, world and local distribution, and erences fechaacceso19 de agosto de 2009 autorWatson L, Dallwitz MJ fecha2008 obraThe Grass Genera of the World / Es originaria de Brasil donde se encuentra en la región del Amazonas (Brasil)Amazonas E

In [8]:
categorylinks_contenido = rdd_category_links.map(lambda x: (x.cl_from,x.cl_to))

## Le asigno una categoria a cada Stub hallado

Lo cacheo para un analisis posterior de la data

In [9]:
stubs_con_category = rdd_stubs.join(categorylinks_contenido).cache()

## Cuento la cantidad de Stubs por categoría.

> Nota: Esto va a tomar un rato pero si carga.

In [10]:
stubs_con_category.map(lambda x: (x[1][1], 1)).reduceByKey(lambda x,y: x+y).collect()

[('1287', 1),
 ('Banderas_con_estrellas', 1),
 ('Accidentes_costeros_y_oceÃ¡nicos', 6),
 ('Gobernantes_por_paÃ\xads', 6),
 ('Sonido', 2),
 ('Wikipedia:ArtÃ\xadculos_con_identificadores_GND', 1116),
 ('Wikipedia:ArtÃ\xadculos_con_identificadores_LCCN', 991),
 ('GÃ©neros_de_danza', 5),
 ('Fauna_por_paÃ\xads', 6),
 ('MÃºsica_de_Cuba', 4),
 ('MÃºsica_del_Caribe', 3),
 ('Alumnado_de_la_Universidad_de_Stuttgart', 1),
 ('Wikipedia:ArtÃ\xadculos_con_identificadores_ISNI', 543),
 ('Wikipedia:ArtÃ\xadculos_con_identificadores_RKDartists', 36),
 ('Organizaciones_polÃ\xadticas_de_CataluÃ±a', 1),
 ('Tipos_de_palabras', 2),
 ('Historia_de_Europa_por_paÃ\xads', 5),
 ('Historia_de_la_UniÃ³n_Europea_por_paÃ\xads', 2),
 ('PeriÃ³dicos_de_Reino_Unido', 2),
 ('Academias_de_Francia', 1),
 ('Estratovolcanes_de_Filipinas', 4),
 ('DinÃ¡mica_quÃ\xadmica', 1),
 ('Poetas_por_paÃ\xads', 8),
 ('PelÃ\xadculas_por_aÃ±o', 11),
 ('Grabado', 2),
 ('Deportistas_en_los_Juegos_OlÃ\xadmpicos_de_Montreal_1976', 2),
 ('BotÃ¡n

# Conclusion

In [11]:
# Determino la cantidad de Stubs hallados.
rdd_stubs.count()

136858

In [12]:
# Busco la categoria mas usada
stubs_con_category.map(lambda x: (x[1][1], 1)).reduceByKey(lambda x,y: x+y)\
                        .reduce(lambda a,b: a if a[1]>b[1] else b)

('Wikipedia:ArtÃ\xadculos_con_datos_por_trasladar_a_Wikidata', 4940)

In [13]:
# Busco la 2da categoria mas usada
stubs_con_category.filter(lambda x: x[1][1]!='Wikipedia:ArtÃ\xadculos_con_datos_por_trasladar_a_Wikidata')\
            .map(lambda x: (x[1][1], 1)).reduceByKey(lambda x,y: x+y)\
                        .reduce(lambda a,b: a if a[1]>b[1] else b)

('Wikipedia:DesambiguaciÃ³n', 2712)

De la muestra dada de textos, observamos que 136858 califican como Stubs y la categoria mas usada para los Stubs es `'Wikipedia:ArtÃ\xadculos_con_datos_por_trasladar_a_Wikidata'`. 

Posiblemente esto sea una etiqueta para saber que ese articulo se debe unir a otro articulo, ya que por ser un Stub naturalmente no clasifica como una definicion propia por lo que hay que resolver que hacer con la misma. Otra de las posibles significados podria ser borrarlo o expandirlo si se puede.

Vemos que la 2da etiqueta mas comun para los Stubs es 'DesambiguaciÃ³n' por lo que es claro que esos 2712 articulos Stub tienen errores y probablemente sean eliminados.